In [1]:
WORKSPACE_PATH = 'workspace'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2

Lade JSON

In [3]:
import json

def json_read(data_path):
    with open(data_path, 'r') as data:
        data = json.load(data)
    return data

In [4]:
JSON_NAME = '/instances_default.json'
DATA_PATH = ANNOTATION_PATH+'/original'+ JSON_NAME
data = json_read(DATA_PATH)

Bennene Bilder im Verzeichnis images und in der Annotationsdatei um

In [5]:
import os

def rename_images(IMAGE_PATH):
    index=1
    img  = os.listdir(IMAGE_PATH)
    for i in img:
        new_name = f"{index:012d}"+'.jpeg'
        old_path = os.path.join(IMAGE_PATH, i)
        new_path = os.path.join(IMAGE_PATH, new_name)
        os.rename(old_path, new_path)
        index += 1

def rename_images_in_annotation():
    index=1
    for img in data['images']:
        new_name = f"{index:012d}"+'.jpeg'
        img['file_name'] = new_name 
        index += 1

rename_images(IMAGE_PATH+'/original')
rename_images_in_annotation()

with open(ANNOTATION_PATH+'/original' + JSON_NAME, 'w') as destination_file:
    json.dump(data, destination_file, indent=2)  

In [6]:
images = []
bounding_boxes = []
labels = []
image_height = 1366
image_width = 768

for image in data['images']:
    one_image = []
    one_image.append(image['id'])
    one_image.append(image['file_name'])
    images.append(one_image)

for annotation in data['annotations']:
    one_box = []
    box_and_id = []
    one_box.append(annotation['bbox'])
    bounding_boxes.append(one_box)

Resize

In [7]:
def resize_imgaes(input_path, output_path, height, width):
    image =  cv2.imread(input_path)
    resized_image = cv2.resize(image, (height, width))
    cv2.imwrite(output_path, resized_image)    

def resize_bounding_boxes_and_area(input_path,bounding_boxes, height, width):
    image =  cv2.imread(input_path)
    height_ratio = height / image.shape[0]
    width_ratio = width / image.shape[1]
    
    for box in bounding_boxes:
        resized_boxes = []
        resized_area = 0

        x = np.round(box[0]*width_ratio,2)
        y = np.round(box[1]*height_ratio,2)
        x_width = np.round(box[2]*width_ratio,2)
        y_height = np.round(box[3]*height_ratio,2)
        resized_boxes.append([x, y, x_width, y_height])

        resized_area = x_width * y_height

    return x, y, x_width, y_height, resized_area


In [8]:
import os

new_height = 224
new_width = 224
OUTPUT_PATH_IMAGES = IMAGE_PATH + '/' + str(new_height) + 'x' + str(new_width)
OUTPUTPATH_ANNOTATIONS = ANNOTATION_PATH + '/' + str(new_height) + 'x' + str(new_width)

os.makedirs(OUTPUT_PATH_IMAGES, exist_ok=True)
os.makedirs(OUTPUTPATH_ANNOTATIONS, exist_ok=True)

#Copy JSON Doc in new ANNOTATIONPATH
with open(DATA_PATH, 'r') as source_file:
    data = json.load(source_file)
with open(OUTPUTPATH_ANNOTATIONS + JSON_NAME, 'w') as destination_file:
    json.dump(data, destination_file, indent=2)    


In [9]:
#Resize Images
for img in range(len(images)):
    resize_imgaes(IMAGE_PATH+'/original'+ '/' +images[img][1], OUTPUT_PATH_IMAGES+ '/' +images[img][1], new_height, new_width)


with open(OUTPUTPATH_ANNOTATIONS + JSON_NAME, 'r') as json_data:
    data = json.load(json_data)


#Resize BBox and area
for i, annotation in enumerate(data['annotations']):
    calculate_resize = resize_bounding_boxes_and_area(IMAGE_PATH+'/original'+ '/' +images[0][1],bounding_boxes[i], new_height, new_width)
    new_box = calculate_resize[0],calculate_resize[1],calculate_resize[2],calculate_resize[3]
    new_area = calculate_resize[4]
    annotation['bbox'] = new_box
    annotation['area'] = new_area

#Change height/width
for image in data['images']:
    image['height']=new_height
    image['width']=new_width

with open(OUTPUTPATH_ANNOTATIONS + JSON_NAME, 'w') as destination_file:
    json.dump(data, destination_file, indent=2)    

Create Dataset

In [20]:
import json

images = data['images']
annotations = data['annotations']
categories = data['categories']

dataset = []

for annotation in annotations:
    image_info = next(image for image in images if image['id'] == annotation['image_id'])
    image_path = OUTPUT_PATH_IMAGES + '/'+ image_info['file_name']
    bbox = annotation['bbox']  # [x, y, width, height]
    class_id = annotation['category_id']

    xmin, ymin, width, height = bbox
    xmax, ymax = xmin + width, ymin + height

    dataset.append({
        'image_path': image_path,
        'bbox': [xmin, ymin, xmax, ymax],
        'class_id': class_id
    })

print('Annotation',len(dataset))
da =dataset[0]
print(dataset[0])
print(da['image_path'])


Annotation 697
{'image_path': 'workspace/images/224x224/000000000001.jpeg', 'bbox': [11.48, 60.11, 151.89, 110.05], 'class_id': 1}
workspace/images/224x224/000000000001.jpeg


In [11]:
import os

def get_file_list(directory):
    file_list = []
    for filename in os.listdir(directory):
        full_path = os.path.join(directory, filename)
        normalized_path = os.path.normpath(full_path)
        normalized_path = normalized_path.replace(os.path.sep, '/')
        if os.path.isfile(normalized_path):
            file_list.append(normalized_path)
    return file_list

In [28]:

def convert_annotations_to_array(coco_data, num_classes=41):
    annotations = coco_data['annotations']
    num_annotations = len(annotations)

    # Initialize empty arrays for bounding boxes and labels
    bounding_boxes = np.zeros((num_annotations, 4), dtype=np.float32)
    labels = np.zeros((num_annotations,), dtype=np.int32)

    for i, annotation in enumerate(annotations):
        # Extract bounding box coordinates
        bbox = annotation['bbox']
        bounding_boxes[i] = np.array([bbox[0], bbox[1], bbox[2], bbox[3]], dtype=np.float32)

        # Extract class label
        labels[i] = annotation['category_id']

    return bounding_boxes, labels


bounding_boxes, labels = convert_annotations_to_array(data)


trainTargets = {
    "class_label": labels,
    "bounding_box": bounding_boxes
}
print(trainTargets)
"""
testTargets = {
    "class_label": testLabels,
    "bounding_box": testBBoxes
}
"""

{'class_label': array([ 1,  2,  3,  3, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
       24, 25, 26, 27, 28, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 39,
       39, 39,  7,  8,  5,  6,  9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
       21, 22, 23, 24, 25, 26, 27, 28, 28, 30, 31, 32, 33, 34, 35, 36, 37,
       38, 39, 39, 39, 39,  7,  8,  5,  6,  9, 10,  2,  3,  4, 11, 12, 13,
       14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 28, 30,
       31, 32, 33, 34, 35, 36, 37, 38, 39, 39, 39, 39,  7,  8,  5,  6,  9,
       10,  2,  3,  4, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
       24, 25, 26, 27, 28, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 39,
       39, 39,  7,  8,  5,  6,  9, 10,  2,  3,  4, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 28, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 39, 39, 39,  7,  8,  5,  6,  9, 10,  2,  3,
        4, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27

'\ntestTargets = {\n    "class_label": testLabels,\n    "bounding_box": testBBoxes\n}\n'

In [24]:
directory_path = OUTPUT_PATH_IMAGES
files = get_file_list(directory_path)


bbox_list=[]
class_id_list=[]
image_path_list=[]

for k in files:
    ann_bbox=[]
    ann_class_id=[]
    for i in dataset:
        d = i
        if d['image_path'] == k:
            ann_bbox.append(d['bbox'])
            #ann_bbox.append(d['class_id'])
            ann_class_id.append(d['class_id'])
    
    image = cv2.imread(d['image_path'])
    image = image / 255.0 

        
    bbox_list.append(ann_bbox)
    class_id_list.append(ann_class_id)
    image_path_list.append(image)

images_data = np.array(image_path_list)
bounding_boxes = np.array(bbox_list)
class_labels = np.array(class_id_list)

print(images_data.shape)
print(bounding_boxes.shape)
print(class_labels.shape)



(17, 224, 224, 3)
(17, 41, 4)
(17, 41)


In [15]:
import keras_cv

base_model = keras_cv.models.MobileNetV3Backbone.from_preset(
    "mobilenet_v3_large_imagenet",
    load_weights=False,
)
#Freeze base model
base_model.trainable = False

Using TensorFlow backend




New Top Layer

In [32]:
import keras

inputs = keras.Input(shape=(new_height, new_width, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)

class_id_output = keras.layers.Dense(41, activation='softmax', name='class_id')(x)

bbox_output = keras.layers.Dense(4, name='bbox')(x)
model = keras.Model(inputs, [class_id_output, bbox_output]) 

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss={'class_id': 'sparse_categorical_crossentropy', 'bbox': 'mean_squared_error'},#Regression für bbox
    metrics={'class_id': 'accuracy'}
)

model.fit(images_data,  {"class_id": labels,"bbox": bounding_boxes}, epochs=20, validation_split=0.2)


Epoch 1/20
1/1 [==============================] - 6s 6s/step - loss: 6528.3394 - class_id_loss: 3.7136 - bbox_loss: 6524.6260 - class_id_accuracy: 0.0000e+00 - val_loss: 7094.2456 - val_class_id_loss: 3.7142 - val_bbox_loss: 7090.5312 - val_class_id_accuracy: 0.0000e+00
Epoch 2/20
1/1 [==============================] - 0s 146ms/step - loss: 6528.2163 - class_id_loss: 3.7122 - bbox_loss: 6524.5044 - class_id_accuracy: 0.1538 - val_loss: 7094.1157 - val_class_id_loss: 3.7147 - val_bbox_loss: 7090.4009 - val_class_id_accuracy: 0.0000e+00
Epoch 3/20
1/1 [==============================] - 0s 142ms/step - loss: 6528.0938 - class_id_loss: 3.7107 - bbox_loss: 6524.3828 - class_id_accuracy: 0.1538 - val_loss: 7093.9849 - val_class_id_loss: 3.7153 - val_bbox_loss: 7090.2695 - val_class_id_accuracy: 0.0000e+00
Epoch 4/20
1/1 [==============================] - 0s 138ms/step - loss: 6527.9697 - class_id_loss: 3.7093 - bbox_loss: 6524.2607 - class_id_accuracy: 0.1538 - val_loss: 7093.8550 - val_clas

In [43]:
img = OUTPUT_PATH_IMAGES + '/000000000001.jpeg'
img = image.load_img(OUTPUT_PATH_IMAGES + '/000000000001.jpeg', target_size=(new_height, new_width))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) 
img_array /= 255.0  
boxPreds, labelPreds = model.predict(img_array)
print(boxPreds)


1/1 [==============================] - 0s 31ms/step
[[0.02410258 0.02508543 0.02508543 0.02508567 0.02410258 0.02410258
  0.02410258 0.02410258 0.02410258 0.02410258 0.02410258 0.02508543
  0.02508543 0.02508543 0.02508543 0.02508543 0.02508543 0.02508543
  0.02508543 0.02508543 0.02410258 0.02410258 0.02410258 0.02410258
  0.02410258 0.02410258 0.02410258 0.02410258 0.02410258 0.02410258
  0.02410258 0.02410258 0.02410258 0.02410258 0.02410258 0.02410258
  0.02410258 0.02410258 0.02410258 0.02410258 0.02410258]]
